In [ ]:
# Initialization

import math
from datetime import datetime
import openpyxl as opxl
from openpyxl.utils.dataframe import dataframe_to_rows
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import model as ty
import proc as pr

import heat_transfer as bht

import os

par = pr.create_par()

pathout,wbo = pr.create_out()
sheet_o = wbo.active
sheet_o.title = "Inputs"

G_list,coeff_G_p_list,u_list,T_amb_list,T_f_in_list,T_guess_list,condi = pr.create_inputs(r'\210914_TUV_test_conditions_SPRING.xlsx')

# Pour titre fichier outputs
moment = str(datetime.now().hour)+'-'+str(datetime.now().minute)+'-'+str(datetime.now().second)

print(par["test"])

In [ ]:
# Test complet TUV ou général

if par["test"] == "TUV":
    df_par,df,X = ty.simu_condi(par,condi)
elif par["test"] == "general": # general
    df_par,df,X = ty.simu_multi_condi(par,G_list,coeff_G_p_list,T_amb_list,u_list,T_guess_list,T_f_in_list)
else:
    print("Erreur de type de test")
    raise KeyboardInterrupt

# Creating Excel outputs

if par["test"] == "TUV":
    prefixe = r'\OutputsTUV'
else: # general
    prefixe = r'\Parametric'

suffixe = prefixe+str(par["ailette"])+r'-'+moment+r'.xlsx'
fichier_o = pathout+suffixe

for d in dataframe_to_rows(df_par, index=True, header=False):
    sheet_o.append(d)

wso = wbo.create_sheet("Outputs")

for d in dataframe_to_rows(df, index=True, header=True):
    wso.append(d)

huit = len(X[0])-1

index_coeff = ['a1','a2','a3','a4','a6','a7','a8']
ws_lsm = wbo.create_sheet("LSM")
ws_lsm['A'+str(1)] = 'a0'
ws_lsm['B'+str(1)] = X[0][len(X[0])-1]

print('a0 : ',round(X[0][len(X[0])-1],3))

ws_lsm['C'+str(1)] = X[3][len(X[0])-1]
for l in range(huit):
    ws_lsm['A'+str(l+2)] = index_coeff[l]
    if l != 3:
        ws_lsm['B'+str(l+2)] = -X[0][l]
        print(index_coeff[l],' : ',round(-X[0][l],3))

    else:
        ws_lsm['B'+str(l+2)] = X[0][l]
        print(index_coeff[l],' : ',round(X[0][l],3))

    ws_lsm['C'+str(l+2)] = X[3][l]

ws_lsm['A'+str(huit+2)] = 'a0 (DS) at 1,3 m/s'
ws_lsm['B'+str(huit+2)] = X[0][len(X[0])-1] - (-X[0][4]*(1.3-3))
print(round(X[0][len(X[0])-1] - (-X[0][4]*(1.3-3)),3)*100,'%')

ws_lsm['A'+str(huit+3)] = 'a1 (DS) at 1,3 m/s'
ws_lsm['B'+str(huit+3)] = -X[0][0] - X[0][2]*(1.3-3)
print(round(-X[0][0] - X[0][2]*(1.3-3),1))
    
wbo.save(filename = fichier_o)
wbo.close()

print(df)

print("Finished")

In [ ]:
path = 'c:\\Users\\valen\\Documents'
inp = r'\insulated_meander_TUV.csv'
TUV = pd.read_csv(path+inp,sep=";",decimal=',')
TUV['Qdot / (A_G x G)'] = TUV['Qdot/A_G']/TUV['G']

plt.scatter(TUV[TUV.U==0.7]['tin'],TUV[TUV.U==0.7]['Qdot / (A_G x G)'],label = "TUV",color='blue')
plt.errorbar(TUV[TUV.U==0.7]['tin'],df[df.u==0.7]['Q_dot / (A_G x G)'],yerr=0.05*np.array(df[df.u==0.7]['Q_dot / (A_G x G)']),fmt='o',label = "1D model",color='red')

plt.scatter(TUV[TUV.U==1.4]['tin'],TUV[TUV.U==1.4]['Qdot / (A_G x G)'],color='blue')
plt.errorbar(TUV[TUV.U==1.4]['tin'],df[df.u==1.4]['Q_dot / (A_G x G)'],yerr=0.05*np.array(df[df.u==1.4]['Q_dot / (A_G x G)']),fmt='o',color='red')

plt.scatter(TUV[TUV.U==2.7]['tin'],TUV[TUV.U==2.7]['Qdot / (A_G x G)'],color='blue')
plt.errorbar(TUV[TUV.U==2.7]['tin'],df[df.u==2.7]['Q_dot / (A_G x G)'],yerr=0.05*np.array(df[df.u==2.7]['Q_dot / (A_G x G)']),fmt='o',color='red')

plt.xlabel('T_fluid,in (°C)')
plt.ylabel('P_th / (A_G x G)')

plt.legend()

# save the plot as a file
plt.savefig('TUV_vs_model_net.png',format='png',dpi=200,bbox_inches='tight')

plt.show()

In [ ]:
# Parametric study

test_list = np.linspace(25,125,20)

A0 = []
A1 = []
A2 = []
A3 = []
A4 = []
A6 = []
A7 = []
A8 = []

for i in range(len(test_list)):

    pr.proc(par,par["test"],i,test_list)

    # Model
    # if 'TUV' in par["test"]:
    #     df_par,df,X = ty.simu_condi(par,condi)
    # else:
    #     df_par,df,X = ty.simu_multi_condi(par,G_list,coeff_G_p_list,T_amb_list,u_list,T_guess_list,T_f_in_list)

    df_par,df,X = ty.simu_condi(par,condi)

    A0.append(X[0][len(X[0])-1])
    A1.append(-X[0][0])
    A2.append(-X[0][1])
    A3.append(-X[0][2])
    A4.append(X[0][3])
    A6.append(-X[0][4])
    A7.append(-X[0][5])
    A8.append(-X[0][6])

# Creating Excel outputs

suffixe = r'\Parametric'+str(par["ailette"])+r'-'+moment+r'.xlsx'
fichier_o = pathout+suffixe

for d in dataframe_to_rows(df_par, index=True, header=False):
    sheet_o.append(d)

wso = wbo.create_sheet("Outputs")

wso['A'+str(1)] = par["test"]
wso['B'+str(1)] = 'a0'
wso['C'+str(1)] = 'a1'
wso['D'+str(1)] = 'a2'
wso['E'+str(1)] = 'a3'
wso['F'+str(1)] = 'a4'
wso['G'+str(1)] = 'a6'
wso['H'+str(1)] = 'a7'
wso['I'+str(1)] = 'a8'
wso['J'+str(1)] = 'a0 DS'
wso['K'+str(1)] = 'a1 DS'

A0DS = []
A1DS = []

for l in range(len(test_list)):
    wso['A'+str(l+2)] = test_list[l]
    wso['B'+str(l+2)] = A0[l]
    wso['C'+str(l+2)] = A1[l]
    wso['D'+str(l+2)] = A2[l]
    wso['E'+str(l+2)] = A3[l]
    wso['F'+str(l+2)] = A4[l]
    wso['G'+str(l+2)] = A6[l]
    wso['H'+str(l+2)] = A7[l]
    wso['I'+str(l+2)] = A8[l]
    wso['J'+str(l+2)] = A0[l]-A6[l]*(-1.7)
    wso['K'+str(l+2)] = A1[l]+A3[l]*(-1.7)

    A0DS.append(A0[l]-A6[l]*(-1.7))
    A1DS.append(A1[l]+A3[l]*(-1.7))

wbo.save(filename = fichier_o)
wbo.close()

print("Finished")

In [ ]:
# create figure and axis objects with subplots()
fig,ax = plt.subplots()
# make a plot

ax.plot(np.array(test_list),A0DS,
        color="red", 
       marker="o")

# set x-axis label

ax.set_xlabel("Number of fins", fontsize = 14)

# set y-axis label

ax.set_ylabel("A0 at u = 1.3 m/s",
              color="red",
              fontsize=14)

ax.set_ybound(0,1)

# ax.axhline(y = 0.999*0.396177,label='98% of the maximum performance',color='green')
# ax.axvline(x = 0.4, label = 'Thickness of the tested prototype',color='black')
# ax.axhline(y = 0.394,color='black')
# ax.axvline(x = 1.3,color='green')
# ax.legend()

# twin object for two dif       ferent y-axis on the sample plot
ax2=ax.twinx()
# make a plot with different y-axis using second axis object
ax2.plot(np.array(test_list),A1DS,color="blue",marker="o")
ax2.set_ylabel("A1 at u = 1.3 m/s",color="blue",fontsize=14)

# save the plot as a file
plt.savefig('parametric.png',format='png',dpi=200,bbox_inches='tight')

plt.show()

In [ ]:
print(test_list)
print(A0DS[5])

In [ ]:
np.max(A0DS)

In [ ]:
par["h_fluid"]

In [ ]:
# Test d'un panneau dans un set de conditions donné (résultats en détails tranche par tranche)

suffixe = r'\OnePanel-'+moment+r'.xlsx'
fichier_o = pathout+suffixe
var = {}

list_var,list_var_conv = ty.simu_one_steady_state(par,var,par["N_meander"],par["T_fluid_in0"],315,"all")

compt = 2

*var_names0, = var
var_names = ['T_fluid_in'] + var_names0

for m in range(len(var_names)):
    sheet_o.cell(row=1,column=m+1,value = var_names[m])

for l in range(par["N_meander"]):
    for m in range(len(var_names)):
        sheet_o.cell(row=l+2,column=m+1,value = list_var[l][var_names[m]])

sheet_o3 = wbo.create_sheet("Convergence")

var_names2 = ['Slice','T_fluid_in'] + var_names0

for m in range(len(var_names2)):
    sheet_o3.cell(row=1,column=m+1,value = var_names2[m])

for l in range(len(list_var_conv)):
    for m in range(len(var_names2)):
        sheet_o3.cell(row=l+2,column=m+1,value = list_var_conv[l][var_names2[m]])


sheet_o2 = wbo.create_sheet("T_abs")

x_list = np.linspace(0,par["delta"],100)

T_abs = []

delta = par["delta"]

for k in range(par["N_meander"]):
    b = list_var[k]["b"]
    j = list_var[k]["j"]
    m = list_var[k]["m"]
    T_B = list_var[k]["T_Base_mean"]

    b_j = b/j

    for l in range(len(x_list)):
        res = b_j+((T_B-b_j)/math.cosh(m*delta))*math.cosh(m*x_list[l])

        sheet_o2.cell(row=l+1,column=k+2,value = res)

for l in range(len(x_list)):
    sheet_o2.cell(row=l+1,column=1,value=x_list[l])

wbo.save(filename = fichier_o)
wbo.close()

print("Finished")

In [ ]:
# Test heat transfer coefficient

T_abs_list = np.array([15,17,19,21,23,25,27,29,31,33,35])
T_abs_list = T_abs_list + np.array(len(T_abs_list)*[273.15])

T_amb = 25+273.15

DT = []
htc = []
htc_top = []
for i in range(len(T_abs_list)):
    DT.append(T_abs_list[i]-T_amb)
    htc.append(bht.back_h_simple(T_abs_list[i],T_amb,30,1.6))
    htc_top.append(bht.top_h_simple(T_abs_list[i],T_amb,30,1.6))

plt.plot(DT,htc,label="Upper side")
plt.plot(DT,htc_top,label="Lower side")
plt.legend()
plt.show()